Giải mã Mê Cung (Maze Solving) là ví dụ kinh điển nhất để biến lý thuyết Đồ thị thành ứng dụng thực tế "nhìn thấy được".

Chúng ta sẽ sử dụng chiến thuật **BFS (Vết Dầu Loang)**.
**Lý do:** Trong mê cung, chúng ta thường muốn tìm **đường thoát ngắn nhất**. Nếu dùng DFS, có thể bạn sẽ đi lòng vòng một con đường rất xa mới tới đích.

---

# 1. Tư duy: Biến Mê Cung thành Đồ thị 🗺️

Đừng nghĩ mê cung là hình ảnh. Hãy nghĩ nó là một bảng số liệu (Matrix/Grid 2D).

* **Đỉnh (Node):** Mỗi ô vuông trống `(hàng, cột)` là một đỉnh.
* **Cạnh (Edge):** Nếu hai ô nằm cạnh nhau (Trên, Dưới, Trái, Phải) và không bị chặn bởi tường, thì có đường nối giữa chúng.

Ví dụ Mê cung 5x5:

* `S`: Start (Bắt đầu)
* `E`: End (Đích)
* `#`: Tường (Không đi được)
* `     ` (Khoảng trắng): Đường đi

```text
                                    
      0     1     2     3     4
    ┌─────┬─────┬─────┬─────┬─────┐───────▶ Hướng y tăng
  0 │  S  │     │  #  │     │     │ 
    ├─────┼─────┼─────┼─────┼─────┤
  1 │     │  #  │     │  #  │     │
    ├─────┼─────┼─────┼─────┼─────┤
  2 │     │     │     │     │     │
    ├─────┼─────┼─────┼─────┼─────┤
  3 │  #  │  #  │  #  │  #  │     │
    ├─────┼─────┼─────┼─────┼─────┤
  4 │     │     │     │  E  │     │
    └─────┴─────┴─────┴─────┴─────┘
    │
    ↓
Hướng x tăng

```


# 2. Chiến thuật Code: BFS trên Lưới 2D

Khác với đồ thị `A, B, C` ta liệt kê bằng tay, với lưới 2D ta dùng toạ độ `(x, y) = (column_index, row_index)`:

1. **Hàng đợi (Queue):** Lưu toạ độ ô đang đứng `[(0,0), ...]`.
2. **Hướng đi (Directions):** Tại một ô, ta có thể đi 4 hướng. Ta dùng mảng kỹ thuật để tính toạ độ hàng xóm:
* Lên: `(x-1, y)`
* Xuống: `(x+1, y)`
* Trái: `(x, y-1)`
* Phải: `(x, y+1)`


3. **Truy vết đường đi (Parent Map):** Để vẽ lại đường đi sau khi tìm thấy đích, ta cần nhớ "Mình đến ô này từ ô nào?".
* Ví dụ: `parent[(0,1)] = (0,0)` (Tôi đến ô (0,1) từ ô (0,0)).




# 3. Code Python "Giải Mê Cung" 🐍

Dưới đây là đoạn code hoàn chỉnh, mình đã comment rất kỹ từng bước.


In [4]:
from collections import deque

def solve_maze_bfs(maze, start, end):
    rows = len(maze)
    cols = len(maze[0])
    
    # 1. Khởi tạo
    queue = deque([start])      # Hàng đợi chứa toạ độ (row, col)
    visited = set([start])      # Sổ tay các ô đã đi
    parent_map = {start: None}  # Lưu vết: parent[con] = cha (để dò ngược đường về)
    
    # Kỹ thuật di chuyển 4 hướng (Lên, Xuống, Trái, Phải)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    # 2. Vòng lặp BFS (Vết dầu loang)
    while queue:
        current_r, current_c = queue.popleft() # Lấy ô đầu hàng đợi
        
        # Nếu chạm đích -> DỪNG NGAY
        if (current_r, current_c) == end:
            print(f"Đã đến đích! Các bước đi: {parent_map}")
            return reconstruct_path(parent_map, end)
        
        # Xét 4 ô xung quanh
        for dr, dc in directions:
            next_r, next_c = current_r + dr, current_c + dc
            
            # Kiểm tra hợp lệ:
            # a. Nằm trong bản đồ
            # b. Không phải tường (#)
            # c. Chưa từng ghé thăm
            if (0 <= next_r < rows and 0 <= next_c < cols and 
                maze[next_r][next_c] != '#' and 
                (next_r, next_c) not in visited):
                
                # Hợp lệ thì thêm vào hàng đợi
                queue.append((next_r, next_c))
                visited.add((next_r, next_c))
                parent_map[(next_r, next_c)] = (current_r, current_c) # Ghi nhớ cha
                
    return None # Không tìm thấy đường

# Hàm phụ: Dò ngược từ Đích về Start để vẽ đường đi
def reconstruct_path(parent_map, current):
    path = []
    while current is not None:
        path.append(current)
        current = parent_map[current]
    return path[::-1] # Đảo ngược lại để có từ Start -> End

# --- CHẠY THỬ ---

# Mê cung (0 là đường, 1 là tường)
maze_grid = [
    ['S', ' ', '#', ' ', ' '],
    [' ', '#', ' ', '#', ' '],
    [' ', ' ', ' ', ' ', ' '],
    ['#', '#', '#', '#', ' '],
    [' ', ' ', ' ', 'E', ' ']
]

start_pos = (0, 0) # Góc trên trái
end_pos = (4, 3)   # Chữ E

path = solve_maze_bfs(maze_grid, start_pos, end_pos)

if path:
    print(f"🎉 Tìm thấy đường thoát ngắn nhất sau khi dò ngược ({len(path)} bước):")
    print(path)
    
    # Vẽ đường đi lên bản đồ cho dễ nhìn
    for r, c in path:
        if maze_grid[r][c] not in ('S', 'E'):
            maze_grid[r][c] = '●' # Dấu chân
            
    for row in maze_grid:
        print(" ".join(row))
else:
    print("💀 Không có lối thoát!")

Đã đến đích! Các bước đi: {(0, 0): None, (1, 0): (0, 0), (0, 1): (0, 0), (2, 0): (1, 0), (2, 1): (2, 0), (2, 2): (2, 1), (1, 2): (2, 2), (2, 3): (2, 2), (2, 4): (2, 3), (1, 4): (2, 4), (3, 4): (2, 4), (0, 4): (1, 4), (4, 4): (3, 4), (0, 3): (0, 4), (4, 3): (4, 4)}
🎉 Tìm thấy đường thoát ngắn nhất sau khi dò ngược (10 bước):
[(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4), (4, 3)]
S   #    
● #   #  
● ● ● ● ●
# # # # ●
      E ●



# 4. Kết quả chạy ("Chạy bằng cơm" qua hình ảnh) 🏃

Khi bạn chạy đoạn code trên, BFS sẽ lan ra như sau:

**Trạng thái ban đầu:**

```text
S   #    
  #   #  
         
# # # #  
      E  

```

**Quá trình BFS lan toả (tìm kiếm):**
Nó sẽ thử đi xuống `(1,0)` và sang phải `(0,1)`... cứ thế lan dần ra các ô trống.

**Kết quả cuối cùng (Path Reconstruction):**
Máy tính sẽ tìm được con đường ngắn nhất và đánh dấu bằng `●`:

```text
S ● #     
  # ● #   
● ● ●     
# # # # ● 
      E ● 

```



# 5. Tại sao bài này lại hay? 💡

Bài toán này giúp bạn hiểu 2 kỹ thuật cực quan trọng của lập trình viên:

1. **Dùng mảng `directions`:** Thay vì viết 4 câu lệnh `if` dài dòng (if lên, if xuống...), ta dùng vòng lặp `for dr, dc in directions`. Code gọn và chuyên nghiệp hơn hẳn.
2. **Truy vết (Traceback):** BFS chỉ giúp ta đi đến đích. Nhưng để trả lời câu hỏi "Đi đường nào?", ta phải dùng `parent_map` để lần ngược dấu chân từ Đích về Xuất phát.

---


# Giải đáp thắc mắc: Tại sao `(0, 1)` thỏa mãn điều kiện nhưng "bay màu" khỏi `path`?

Bạn nói đúng:

1. `(0, 1)` **Nằm trong bản đồ**.
2. `(0, 1)` **Không phải tường** (là khoảng trống).
3. `(0, 1)` **Chưa từng ghé thăm** (lúc mới bắt đầu).

👉 **Sự thật là:** Máy tính **CÓ** ghé thăm `(0, 1)`, nó **CÓ** thêm `(0, 1)` vào hàng đợi và **CÓ** ghi vào `parent_map` (với cha là `(0, 0)`).

Tuy nhiên, lý do nó không xuất hiện trong kết quả cuối cùng (`path`) là vì: **Nó là một ngõ cụt (Dead End).**

Hãy cùng "chạy bằng cơm" (Trace code) đoạn này để thấy rõ số phận hẩm hiu của ô `(0, 1)` nhé.



## Bản đồ (Zoom vào góc 3x3)

```text
(0,0)S  (0,1)   (0,2)#
(1,0)   (1,1)#  (1,2) 
(2,0)   (2,1)   (2,2)

```

*(Lưu ý: `#` là tường)*



## Bước 1: Tại `(0, 0)` [Start]

* Máy tính nhìn xung quanh và thấy 2 ô hợp lệ:
1. `(1, 0)` (Ô bên dưới).
2. `(0, 1)` (Ô bên phải).


* Nó thêm cả 2 vào Hàng đợi.
* **Sổ cái (Parent Map):**
* `parent[(1, 0)] = (0, 0)`
* `parent[(0, 1)] = (0, 0)`





## Bước 2: Xử lý nhánh `(0, 1)`

Đến lượt ô `(0, 1)` được lấy ra khỏi hàng đợi. Máy tính đứng tại đây và tìm đường đi tiếp:

* **Nhìn sang Phải `(0, 2)`:** Gặp Tường `#` ❌ -> Bỏ.
* **Nhìn xuống Dưới `(1, 1)`:** Gặp Tường `#` ❌ -> Bỏ.
* **Nhìn sang Trái `(0, 0)`:** Đã thăm rồi (`visited`) ❌ -> Bỏ.
* **Nhìn lên Trên:** Hết bản đồ ❌ -> Bỏ.

👉 **Kết luận:** Ô `(0, 1)` không đẻ ra được bước đi tiếp theo nào cả. Nhánh đường này **CHẾT** tại đây. Không ai đi tiếp từ `(0, 1)` đến đích được.



## Bước 3: Xử lý nhánh `(1, 0)`

Đến lượt ô `(1, 0)` được lấy ra.

* **Nhìn xuống Dưới `(2, 0)`:** Ngon lành! Khoảng trống, chưa thăm.
* Nó thêm `(2, 0)` vào hàng đợi.
* **Sổ cái (Parent Map):** `parent[(2, 0)] = (1, 0)`.

Từ đây, nhánh này cứ thế phát triển tiếp: `(2, 0) -> (2, 1) -> ... -> Đích`.



## Bước 4: Truy vết (Reconstruct Path) - Sự thật phơi bày

Khi tìm thấy Đích, hàm `reconstruct_path` sẽ dò ngược từ Đích về Start dựa trên `parent_map`.

Hãy xem "gia phả" của con đường thành công:

* Đích `E` là con của ông `X`.
* ...
* Ông `(2, 0)` là con của ông `(1, 0)`.
* Ông `(1, 0)` là con của ông `(0, 0)`.

**Còn ô `(0, 1)` nằm ở đâu?**
Nó nằm trong `parent_map` (`(0, 1)` là con của `(0, 0)`), nhưng **nó không có đứa con nào dẫn đến Đích cả**.
Do đó, khi dò ngược từ Đích về, chúng ta không bao giờ đi ngang qua `(0, 1)`.


## Tóm tắt bằng hình ảnh

Tưởng tượng cái cây gia phả như thế này:

```text
            [ (0,0) START ]
            /             \
        [ (1,0) ]       [ (0,1) ]
           ↓               ❌
        [ (2,0) ]       (Ngõ cụt: Bị tường chặn hết)
           ↓
        [ (2,1) ]
           ↓
          ...
           ↓
        [ ĐÍCH ]

```

* **Có duyệt `(0, 1)` không?** CÓ.
* **Nằm trong đường ngắn nhất (`path`) không?** KHÔNG. Vì nó không dẫn đến đâu cả.

Bạn thấy đấy, thuật toán rất công bằng: Cho cơ hội ("thăm") tất cả mọi người, nhưng chỉ vinh danh ("in ra path") những người có đóng góp dẫn đến thành công thôi! 😄